In [ ]:
import os
os.environ['NEWS_API_KEY'] = 'bca996d07c1842d7846eb24ae94d8bea'


In [ ]:
!pip install requests pandas


In [ ]:
import requests
import pandas as pd
import os
from google.colab import drive

# Ensure the drive is mounted
drive.mount('/content/drive')

def fetch_articles(api_key, from_date, to_date, language='en', total_articles=15):
    print("Fetching articles...")
    url = 'https://newsapi.org/v2/everything'
    articles = []
    pageSize = 30  # Maximum reasonable pageSize considering the API limits
    totalPages = (total_articles + pageSize - 1) // pageSize
    query = 'news'  # Using a general keyword to fetch broad results

    for page in range(1, totalPages + 1):
        params = {
            'q': query,  # Added a generic query for broader results
            'from': from_date,
            'to': to_date,
            'language': language,
            'pageSize': pageSize,
            'page': page,
            'apiKey': api_key
        }
        try:
            response = requests.get(url, params=params)
            response.raise_for_status()  # Will raise an exception for 4XX/5XX errors
            data = response.json()
            articles.extend(data['articles'])
            if len(articles) >= total_articles:
                break
        except requests.exceptions.RequestException as e:
            print(f"Failed to fetch articles due to: {e}")
            break

    return articles[:total_articles]

def save_articles_to_csv(articles, path):
    df = pd.DataFrame(articles)
    df.to_csv(path, index=False)
    print(f"File saved at: {path}")

def get_file_path(filename):
    return f'/content/drive/My Drive/GoogleColab/news_articles{filename}'

def main():
    api_key = os.getenv('NEWS_API_KEY')  # Check that your environment variable is properly set
    from_date = '2024-04-29'
    to_date = '2024-05-07'
    path = get_file_path('articles.csv')

    articles = fetch_articles(api_key, from_date, to_date)

    if articles:
        articles_data = [{'title': article['title'], 'content': article.get('content', 'No content available')} for article in articles]
        save_articles_to_csv(articles_data, path)
        print(f"Saved {len(articles_data)} articles to '{path}'")
    else:
        print("No articles found or fetched.")

if __name__ == '__main__':
    main()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Fetching articles...
File saved at: /content/drive/My Drive/GoogleColab/api_news_articlesarticles.csv
Saved 30 articles to '/content/drive/My Drive/GoogleColab/api_news_articlesarticles.csv'
